#17. Transform Orders Data - Explode Arrays
1. Access Element from JSON objects
2. deduplicate Array Elements
3. Explode Arrays
4. Write transformed data to silver schema

###1. Access element from JSON objects

In [0]:
%sql
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       json_value.items
FROM shoppix.silver.orders_json

###Deduplicate Array Elements

In [0]:
%sql
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       array_distinct(json_value.items) AS items
FROM shoppix.silver.orders_json

###3. Explode Arrays

In [0]:
%sql
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       explode(array_distinct(json_value.items)) AS items
FROM shoppix.silver.orders_json

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW tv_orders_exploded
AS
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       explode(array_distinct(json_value.items)) AS items
FROM shoppix.silver.orders_json
       

In [0]:
%sql
SELECT order_id,
       order_status,
       payment_method,
       total_amount,
       transaction_timestamp,
       customer_id,
       items.item_id,
       items.name,
       items.price,
       items.quantity,
       items.category,
       items.details.brand,
       items.details.color
FROM tv_orders_exploded;
       

###Write transformed data to silver schema

In [0]:
%sql
CREATE TABLE IF NOT EXISTS shoppix.silver.orders
AS
SELECT order_id,
       order_status,
       payment_method,
       total_amount,
       transaction_timestamp,
       customer_id,
       items.item_id,
       items.name,
       items.price,
       items.quantity,
       items.category,
       items.details.brand,
       items.details.color
FROM tv_orders_exploded;